iMPORTS

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import os
from deep_translator import GoogleTranslator

ModuleNotFoundError: No module named 'requests'

Utils

In [8]:
def translation(source='pl', target='en'):
   return GoogleTranslator(source,target).translate(text)

In [2]:
def extract(ancestor, selector=None, attribute=None, multiple=False):
   if selector:
      if multiple:
         if attribute:
          return [tag[attribute].strip() for tag in ancestor.select(selector)]
         return[tag.text.strip() for tag in ancestor.select(selector)]
      if attribute:
         try:
            return ancestor.select_one(selector)[attribute].strip()
         except AttributeError:
            return None
      try:
         return ancestor.select_one(selector).text.strip()
      except AttributeError:
         return None
   if attribute:
      return ancestor[attribute]
   return ancestor.text.strip()

In [3]:
selectors = {
'opinion_id':("data-entry-id",),
'author':("span.user-post__author-name",),
'recommend':("span.user-post__author-recomendation> em",),
'stars':("span.user-post__score-count",),
'content':("div.user-post__text",),
'pros':("div.review-feature__item--positive",None,True),
'cons':("div.review-feature__item--negative",None,True),
'up_votes':("button.vote-yes","data-total-vote"),
'down_votes':("button.vote-no","data-total-vote"),
'published':("span.user-post__published > time:nth-child(1)","datetime"),
'purshaped':("span.user-post__published > time:nth-child(2)","datetime")
}

In [4]:
with open("./cookie.json","r",encoding="UTF-8") as jf:
    headers = json.load(jf)

1. send a request for accessing first webpage with opinions about products


2. If responce is OK, parse HTML page content into DOM structure 

3. Extract from DOM structure  opinions and their components

4. Assign opinions with their cmponents to complex data structures


5. If there are more pages with opinions, repeat all steps 1-5

In [ ]:
product_id = 169771365
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        print(url)
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlights)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion =  {
                key: extract(opinion, *values)
                for key, values in selectors.items()
            }
            all_opinions.append(single_opinion)
           
        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination_next")["href"]
        except TypeError:
            url = None

In [6]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [7]:
with open (f"./opinions/{product_id}.json","w", encoding="UTF-8" ) as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)  